In [ ]:
import pandas as pd
import matplotlib as plt #.pyplot
import numpy as np
import requests

In [ ]:
from api_key_coingecko import API_KEY

url = f"https://api.coingecko.com/api/v3/coins/bitcoin/market_chart" #api end point 

days_to_analyse = 30
currency = 'usd'
headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": API_KEY
}

params = {
    'vs_currency' : currency,
    'days' : str(days_to_analyse)
}

response = requests.get(url, params=params, headers=headers)
data = response.json()


In [ ]:
import nbimporter
from IPython import get_ipython
import test_coingecko_with_jupyter  # without `.ipynb` extension

#load_tocken_prices(days = 30, currency = 'usd', token = 'bitcoin')